# Metaprogramming with Julia

## Scare Quotes

Julia supports *metaprogramming*. This is similar to symbolic programming, where we deal with expressions (like $2+2$) as opposed to values (like $4$).

Normally, Julia takes all code we give it as a set of instructions, and carries them out. If we type `2+2` it will faithfully add those numbers and give us the result.

In [1]:
2+2

4

We can prevent this from happening with quotation marks. Surrounding our code with `"` treats it as a literal string of characters, without seeing it as code at all.

In [2]:
x = "2+2"

"2+2"

We can then explicitly tell Julia to evaluate it later on.

In [3]:
eval(Meta.parse(ans))

4

Why go through this complicated way to add numbers? The trick is that when we have the expression `2+2`, we can modify it in various interesting ways. As a simple example, imagine replacing `+` with `-`.

In [4]:
x = replace(x, "+"=>"-")

"2-2"

In [5]:
eval(Meta.parse(x))

0

We don't actually want to work with strings here; Julia has a much more powerful way to quote code, the frowny face operator `:()`.

In [6]:
x = :(2+2)

:(2 + 2)

In [7]:
eval(x)

4

We can quote larger expression, including blocks and entire function definitions. The keyword `quote` is an alternative to `begin` that returns the quoted block.

In larger blocks, Julia will preserve line number information, which appears as a comment.

In [30]:
quote
  x = 2 + 2
  hypot(x, 5)
end

quote
    #= In[30]:2 =#
    x = 2 + 2
    #= In[30]:3 =#
    hypot(x, 5)
end

In [42]:
?hypot

search: hypot



```
hypot(x, y)
```

Compute the hypotenuse $\sqrt{|x|^2+|y|^2}$ avoiding overflow and underflow.

This code is an implementation of the algorithm described in: An Improved Algorithm for `hypot(a,b)` by Carlos F. Borges The article is available online at ArXiv at the link   https://arxiv.org/abs/1904.09481

```
hypot(x...)
```

Compute the hypotenuse $\sqrt{\sum |x_i|^2}$ avoiding overflow and underflow.

# Examples

```jldoctest; filter = r"Stacktrace:(\n \[[0-9]+\].*)*"
julia> a = Int64(10)^10;

julia> hypot(a, a)
1.4142135623730951e10

julia> √(a^2 + a^2) # a^2 overflows
ERROR: DomainError with -2.914184810805068e18:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).
Stacktrace:
[...]

julia> hypot(3, 4im)
5.0

julia> hypot(-5.7)
5.7

julia> hypot(3, 4im, 12.0)
13.0
```


In [62]:
:(function mysum(count)
    sum = 0
    for x in 1:count
      sum += x
    end
end)

:(function mysum(count)
      #= In[62]:1 =#
      #= In[62]:2 =#
      sum = 0
      #= In[62]:3 =#
      for x = 1:count
          #= In[62]:4 =#
          sum += x
      end
  end)

## Fruit of the Expression Tree

Strings support "interpolation", which allows us to easily build larger strings from smaller components.

In [65]:
x = "yields falsehood when preceded by its quotation"
println(x)

yields falsehood when preceded by its quotation


In [67]:
y = "$x $x"
println(y)

yields falsehood when preceded by its quotation yields falsehood when preceded by its quotation


In [68]:
x = :(2+2)

:(2 + 2)

In [69]:
:($x * $x)

:((2 + 2) * (2 + 2))

In [70]:
eval(ans)

16

## The Root of all Eval

`eval` can do more than just returning a result. What happens if we quote something like a function definition?

In [71]:
ex = :(foo() = println("I'm foo!"))

:(foo() = begin
          #= In[71]:1 =#
          println("I'm foo!")
      end)

It doesn't actually do anything; yet.

In [72]:
foo()  # throws UndefVarError

LoadError: UndefVarError: foo not defined

But evaluating `ex` brings `foo` to life!

In [73]:
eval(ex)

foo (generic function with 1 method)

In [74]:
foo()

I'm foo!


Using interpolation, we can construct a function definition on-the-fly; in fact, we can make a whole series of functions at once.

In [75]:
for name in [:foo, :bar, :baz]
  println(:($name() = println($("I'm $(name)!"))))
end

foo() = begin
        #= In[75]:2 =#
        println("I'm foo!")
    end
bar() = begin
        #= In[75]:2 =#
        println("I'm bar!")
    end
baz() = begin
        #= In[75]:2 =#
        println("I'm baz!")
    end


In [ ]:
dreamcatcher_members = []

And then bring them to life with `eval`, too.

In [20]:
for name in [:foo, :bar, :baz]
  eval(:($name() = println($("I'm $(name)!"))))
end

In [21]:
bar()

I'm bar!


In [22]:
baz()

I'm baz!


This can be an *extremely* useful trick when wrapping APIs (say, from a C library or over HTTP). APIs often define a list of available functions, so you can grab that and generate the whole wrapper at once! See Clang.jl, TensorFlow.jl, or the Base linear algebra wrappers for examples.

## Original sin

Here's a more practical example. Consider the following definition of the `sin` function, based on the Taylor series.

$$sin(x) = \sum_{k=1}^{\infty} \frac{(-1)^k}{(1+2k)!} x^{1+2k}$$

In [76]:
mysin(x) = sum((-1)^k/factorial(1+2k) * x^(1+2k) for k = 0:5)

mysin (generic function with 1 method)

In [77]:
mysin(0.5), sin(0.5)

(0.4794255386041834, 0.479425538604203)

To see where we are right now, we'll benchmark it.

In [78]:
using BenchmarkTools
@benchmark mysin(0.5)

BenchmarkTools.Trial: 10000 samples with 973 evaluations.
 Range (min … max):  72.043 ns … 275.677 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     72.448 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   75.533 ns ±  13.471 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ ▁  ▁                                                       ▁
  ███▇▇█▇▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▆▅▆▆▆▅▅▆▅▅▅▄▅▅▅▄▅▅▅▅▄▅▄▃▄▂▅▄▄▃▄▃▅▂▃▃▄▄ █
  72 ns         Histogram: log(frequency) by time       145 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Right now, this is much slower than it could be. The reason is that we're looping over `k`, which is relatively expensive. It'd be much faster to write out:

In [79]:
mysin(x) = x - x^3/6 + x^5/120 # + ...

mysin (generic function with 1 method)

But this is tedious to write, and no longer looks like the original Taylor series. It's harder to tell if we've made a mistake, and we easily modify it. Is there a way to get the best of both worlds?

How about getting Julia to write out that code for us?

To start with, let's consider a symbolic version of the `+` function.

In [27]:
plus(a, b) = :($a + $b)

plus (generic function with 1 method)

In [28]:
plus(1, 2)

:(1 + 2)

In [80]:
?reduce

search: reduce mapreduce



```
reduce(op, itr; [init])
```

Reduce the given collection `itr` with the given binary operator `op`. If provided, the initial value `init` must be a neutral element for `op` that will be returned for empty collections. It is unspecified whether `init` is used for non-empty collections.

For empty collections, providing `init` will be necessary, except for some special cases (e.g. when `op` is one of `+`, `*`, `max`, `min`, `&`, `|`) when Julia can determine the neutral element of `op`.

Reductions for certain commonly-used operators may have special implementations, and should be used instead: `maximum(itr)`, `minimum(itr)`, `sum(itr)`, `prod(itr)`,  `any(itr)`, `all(itr)`.

The associativity of the reduction is implementation dependent. This means that you can't use non-associative operations like `-` because it is undefined whether `reduce(-,[1,2,3])` should be evaluated as `(1-2)-3` or `1-(2-3)`. Use [`foldl`](@ref) or [`foldr`](@ref) instead for guaranteed left or right associativity.

Some operations accumulate error. Parallelism will be easier if the reduction can be executed in groups. Future versions of Julia might change the algorithm. Note that the elements are not reordered if you use an ordered collection.

# Examples

```jldoctest
julia> reduce(*, [2; 3; 4])
24

julia> reduce(*, [2; 3; 4]; init=-1)
-24
```

---

```
reduce(f, A; dims=:, [init])
```

Reduce 2-argument function `f` along dimensions of `A`. `dims` is a vector specifying the dimensions to reduce, and the keyword argument `init` is the initial value to use in the reductions. For `+`, `*`, `max` and `min` the `init` argument is optional.

The associativity of the reduction is implementation-dependent; if you need a particular associativity, e.g. left-to-right, you should write your own loop or consider using [`foldl`](@ref) or [`foldr`](@ref). See documentation for [`reduce`](@ref).

# Examples

```jldoctest
julia> a = reshape(Vector(1:16), (4,4))
4×4 Matrix{Int64}:
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

julia> reduce(max, a, dims=2)
4×1 Matrix{Int64}:
 13
 14
 15
 16

julia> reduce(max, a, dims=1)
1×4 Matrix{Int64}:
 4  8  12  16
```


With `plus` we can do more interesting things, like symbolic `sum`:

In [29]:
reduce(+, 1:10)

55

In [30]:
reduce(plus, 1:10)

:(((((((((1 + 2) + 3) + 4) + 5) + 6) + 7) + 8) + 9) + 10)

In [31]:
eval(ans)

55

Given that, we can also sum over symbolic variables.

In [32]:
reduce(plus, [:(x^2), :x, 1])

:((x ^ 2 + x) + 1)

This gives us an important piece of the puzzle, but we also need to figure out _what_ we're summing. Let's crate a symbolic version of the Taylor series above, which interpolates the value of `k`.

In [33]:
k = 2
:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k)))

:((1 * x ^ 5) / 120)

Now we have one term, we can generate as many as we like.

In [34]:
terms = [:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k))) for k = 0:5]

6-element Array{Expr,1}:
 :((1 * x ^ 1) / 1)         
 :((-1 * x ^ 3) / 6)        
 :((1 * x ^ 5) / 120)       
 :((-1 * x ^ 7) / 5040)     
 :((1 * x ^ 9) / 362880)    
 :((-1 * x ^ 11) / 39916800)

And sum them –

In [35]:
reduce(plus, ans)

:((((((1 * x ^ 1) / 1 + (-1 * x ^ 3) / 6) + (1 * x ^ 5) / 120) + (-1 * x ^ 7) / 5040) + (1 * x ^ 9) / 362880) + (-1 * x ^ 11) / 39916800)

And create a function definition out of it:

In [36]:
:(mysin(x) = $ans)

:(mysin(x) = begin
          #= In[36]:1 =#
          (((((1 * x ^ 1) / 1 + (-1 * x ^ 3) / 6) + (1 * x ^ 5) / 120) + (-1 * x ^ 7) / 5040) + (1 * x ^ 9) / 362880) + (-1 * x ^ 11) / 39916800
      end)

In [37]:
eval(ans)

mysin (generic function with 1 method)

In [38]:
mysin(0.5), sin(0.5)

(0.4794255386041834, 0.479425538604203)

In [39]:
@benchmark mysin(0.5)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     28.162 ns (0.00% GC)
  median time:      31.374 ns (0.00% GC)
  mean time:        32.995 ns (0.00% GC)
  maximum time:     117.487 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     991

On my machine `sin2` takes about 50 *nano*seconds to run – not bad for a naive implementation. If we challenged a photon to a twenty metre sprint, we'd win!

## Beneath the Expression

It's really just a normal tree data structure, and we can take a look inside it.

In [40]:
ex = :(f(x, y))
ex = Expr(:call, :f, :x, :y)
@show ex.head;
@show ex.args;

ex.head = :call
ex.args = Any[:f, :x, :y]


In our example above, we replace `+` with `-` in a string. In an expression, we can do that by altering the `.args` of the expression.

In [41]:
ex = :(2+2)

:(2 + 2)

In [42]:
ex.args[1] = :-
ex

:(2 - 2)

In [43]:
eval(ex)

0

Note that larger, more complex expressions are a bit trickier than this. They are *nested*, which means that the expression `2+3` is contained inside the larger expression `1 + (2 + 3)`.

In [44]:
ex = :(1 + (2 + 3))

:(1 + (2 + 3))

In [45]:
ex.args

3-element Array{Any,1}:
  :+      
 1        
  :(2 + 3)

A package called MacroTools provides a way to deal with this. It shows us *all* sub-expressions in turn, allowing us to decide how we want to change them. It can be thought of as a bit like a find-and-replace operation. Here's an example that finds all integers in an expression, and increments them.

In [46]:
# import Pkg; Pkg.add("MacroTools")
using MacroTools
using MacroTools: postwalk

In [47]:
postwalk(ex) do x
  x isa Integer ? x+1 : x
end

:(2 + (3 + 4))

To get a feel for what's happening, you can use `@show` to see what `postwalk` sees.

(`@show` is Julia's single most useful feature; if you don't understand what code is doing, throw it in and see what's happening.)

In [48]:
map(x -> @show(x), [1,2,3])

x = 1
x = 2
x = 3


3-element Array{Int64,1}:
 1
 2
 3

In [49]:
postwalk(ex) do x
  @show x
end

x = :+
x = 1
x = :+
x = 2
x = 3
x = :(2 + 3)
x = :(1 + (2 + 3))


:(1 + (2 + 3))

MacroTools also provides tools for *pattern matching* over expressions. `a_ + b_` acts as a template; if the expression provided look like the template, `a` and `b` will match the two things be added. If not, they'll just be `nothing`.

In [50]:
ex = :(2 + 3)
@capture(ex, a_ + b_)

true

In [51]:
a, b

(2, 3)

In [52]:
ex = :(f(2,3))
@capture(ex, a_ + b_)

false

In [53]:
a, b

(nothing, nothing)

We can finally use this to replace _all_ `+`s with `-`s in an expression, rather than just one.

In [54]:
ex = :(3x^2 + (2x + 1))

:(3 * x ^ 2 + (2x + 1))

In [55]:
postwalk(ex) do x
  @capture(x, a_ + b_) || return x
  :($a - $b)
end

:(3 * x ^ 2 - (2x - 1))

## Macro Agressions

You have probably seen macros already – basic features in Julia like `@show`, `@time` and `@inline` are really macros. For basic usage, it's enough to think of them as simple annotations that alter how code is run. But we now know enough to dig into how they work under the hood.

Normal functions never see _code_, only _values_. If we pass `2+2` to a function, it sees `4`.

In [56]:
function foo(x)
  @show x
  return x
end

foo (generic function with 2 methods)

In [57]:
foo(2+2)

x = 4


4

Macros are much like function, but they see _code_ that was passed to them, like the expressions that we saw above. Macros get a chance to manipulate this code and alter how it behaves.

In [58]:
macro foo(x)
  @show x
  return x
end

@foo (macro with 1 method)

In [59]:
@foo(2+2)

x = :(2 + 2)


4

As a simple example, we can replace one of the arguments to `+` and get a different result back.

In [60]:
macro foo(x)
  x.args[2] = 5
  return x
end

@foo (macro with 1 method)

In [61]:
@foo(2+2)

7

MacroTools provides a useful tool, `@expand`, to see what's going on inside a macro; it reveals the code that the macro returns without running it.

In [62]:
@expand @foo(2+2)

:(5 + 2)

You can of course use this on the macros that come with Julia, and it's a good way to learn how they work.

In [63]:
@time 2+2

  0.000001 seconds (4 allocations: 160 bytes)


4

In [64]:
@expand @time 2+2

quote
    #= util.jl:154 =#
    local crab = (Base.gc_num)()
    #= util.jl:155 =#
    local chicken = (Base.time_ns)()
    #= util.jl:156 =#
    local okapi = 2 + 2
    #= util.jl:157 =#
    chicken = (Base.time_ns)() - chicken
    #= util.jl:158 =#
    local dogfish = (Base.GC_Diff)((Base.gc_num)(), crab)
    #= util.jl:159 =#
    (Base.time_print)(chicken, dogfish.allocd, dogfish.total_time, (Base.gc_alloc_count)(dogfish))
    #= util.jl:161 =#
    (Base.println)()
    #= util.jl:162 =#
    okapi
end

Let's do something more advanced. Using the techniques we developed above, we can find-and-replace `+` expressions in code. We'll still do the addition, but we'll also log what we added for debugging purposes.

First, let's make an example expression using quotation.

In [65]:
ex = quote
  s = 0
  for x in xs
    s = s + x
  end
  return s
end

quote
    #= In[65]:2 =#
    s = 0
    #= In[65]:3 =#
    for x = xs
        #= In[65]:4 =#
        s = s + x
    end
    #= In[65]:6 =#
    return s
end

We can use `ex` to develop the code transformation we want, and check that the code does the right thing.}

In [66]:
postwalk(ex) do x
  @capture(x, a_ + b_) || return x
  quote
    println("Adding " * string($a) * " to " * string($b))
    $x
  end
end

quote
    #= In[65]:2 =#
    s = 0
    #= In[65]:3 =#
    for x = xs
        #= In[65]:4 =#
        s = begin
                #= In[66]:4 =#
                println("Adding " * string(s) * " to " * string(x))
                #= In[66]:5 =#
                s + x
            end
    end
    #= In[65]:6 =#
    return s
end

Now we just wrap this in a macro, and we can add it to a normal function definition!

In [67]:
macro log_adds(ex)
  postwalk(ex) do x
    @capture(x, a_ + b_) || return x
    quote
      println("Adding " * string($a) * " to " * string($b))
      $x
    end
  end
end

@log_adds (macro with 1 method)

In [68]:
@log_adds function mysum(xs)
  sum = 0
  for x in xs
    sum = sum + x
  end
  return sum
end

mysum (generic function with 1 method)

In [69]:
mysum(1:10)

Adding 0 to 1
Adding 1 to 2
Adding 3 to 3
Adding 6 to 4
Adding 10 to 5
Adding 15 to 6
Adding 21 to 7
Adding 28 to 8
Adding 36 to 9
Adding 45 to 10


55

Adding and removing `@log_adds` is much nicer than inserting the debug calls by hand, especially if you have lots of `+`s in the code. Perhaps you can extend the macro to support logging other operators, like `-`.

## Generated Functions

_Generated functions_ are a new metaprogramming tool unique to Julia. This section will briefly describe and motivate them, and [this blog post](http://mikeinnes.github.io/2017/08/24/cudanative.html) gives more examples for the interested reader.

Essentially, a generated function is like a macro that operates on _types_ rather than expression trees. If we try to inspect arguments, we'll see their types rather than their values.

In [70]:
@generated function gadd(a, b)
  Core.println("a = $a, b = $b")
  :(a+b)
end

gadd (generic function with 1 method)

In [71]:
gadd(5, 2.5)

a = Int64, b = Float64


7.5

Generated functions become more powerful when working with more complex types. For example, notice how arrays store their rank inside the type, so this is something we can generate code for.

In [72]:
rand(2,2)

2×2 Array{Float64,2}:
 0.618286  0.431531
 0.58246   0.555899

In [73]:
typeof(ans)

Array{Float64,2}

Why might this be useful? If you write code to deal with arrays, you'll notice that you often need a `for` loop for each dimension of the array. If you want to work with a 7D array, you need to write seven nested loops!

In [74]:
function mysum(xs::Array{<:Any,1})
  sum = 0
  for i = 1:length(xs)
    sum += xs[i]
  end
  return sum
end

function mysum(xs::Array{<:Any,2})
  sum = 0
  for i = 1:size(xs,1)
    for j = 1:size(xs, 2)
      sum += xs[i]
    end
  end
  return sum
end

mysum (generic function with 3 methods)

Where other languages simply hard-code a version of each function for vectors, matrices and perhaps 3D arrays, Julia allows us to easily write N-dimensional algorithms by generating nested loops as needed.